In [92]:
import torch 
import torch.nn as nn
import numpy as np
import pandas as pd
import yfinance as yf

In [93]:
tsla = yf.Ticker("tsla")

tsla_hist = tsla.history(period='5y', interval='1d', end='2025-06-13')
# tsla_hist

In [94]:
tsla_hist.drop(columns=['Dividends', 'Stock Splits'], inplace=True)

In [95]:
tsla_hist.index =pd.to_numeric(tsla_hist.index)
tsla_hist.index = tsla_hist.index/(max(tsla_hist.index)) + 1

In [96]:
for col in tsla_hist.columns:
    tsla_hist[col] = tsla_hist[col]/max(tsla_hist[col]) + 1

In [97]:
X = tsla_hist.drop(columns=['Close'])
y = tsla_hist['Close']

In [98]:
X['Date'] = X.index

In [99]:
for delay in range(1,4):
    X[f"Delay {delay}"] = y.iloc[(3-delay):-(delay)]

In [100]:
X_sample = [[d, h, l, o, vol, v1, v2, v3] for d, h, l, o, vol, v1, v2, v3 in zip(X['Date'].iloc[3:].values, X['High'].iloc[3:].values, X['Low'].iloc[3:].values,X['Open'].iloc[3:].values, X['Volume'].iloc[3:].values, X['Delay 1'].dropna().values, X['Delay 2'].dropna().values, X['Delay 3'].dropna().values)]
y_sample = y.iloc[3:].values

In [101]:
X_df = pd.DataFrame(X_sample)

In [102]:
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
print(X_df)

             0         1         2         3         4         5         6  \
0     1.910205  1.134548  1.136558  1.133679  1.208316  1.133490  1.139177   
1     1.910254  1.135779  1.139140  1.139354  1.199321  1.136982  1.133490   
2     1.910402  1.137826  1.138215  1.135744  1.203182  1.133336  1.136982   
3     1.910451  1.148427  1.146260  1.140996  1.380831  1.140228  1.133336   
4     1.910501  1.154928  1.157446  1.151713  1.299985  1.150017  1.140228   
5     1.910550  1.167574  1.172761  1.171112  1.388295  1.155550  1.150017   
6     1.910748  1.188015  1.184483  1.178846  1.463023  1.167918  1.155550   
7     1.910797  1.195071  1.194780  1.196821  1.483727  1.190553  1.167918   
8     1.910846  1.193401  1.191084  1.196820  1.367163  1.193092  1.190553   
9     1.910896  1.192214  1.196904  1.195698  1.263760  1.189761  1.193092   
10    1.910945  1.211367  1.200507  1.195559  1.525323  1.193707  1.189761   
11    1.911093  1.244946  1.214365  1.232402  1.877551  1.214597

In [103]:
len(X['Date'].iloc[3:].values), len(X_sample)

(1252, 1252)

In [104]:
goods = []
for samples in X_sample:
    for sample in samples:
        if np.isnan(sample):
            print("contains nan")
        elif sample ==0:
            print("contains 0")
        elif sample != None:
            goods.append(sample)

In [105]:
len(goods)

10016

In [106]:
len(y_sample), len(X_sample)

(1252, 1252)

In [107]:
torch.set_default_dtype(torch.float64)

In [108]:
X = torch.from_numpy(np.array(X))
y = torch.from_numpy(np.array(y))

In [109]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [110]:
len(X_test)

251

In [111]:
train_data = [(X,y) for X, y in zip(X[:-273], y[:-273])]
test_data = [(X,y) for X, y in zip(X[-273:], y[-273:])]

In [112]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(train_data, batch_size=32, shuffle=True)
test_dataloaader= DataLoader(test_data, batch_size=32, shuffle=True)

In [113]:
count = 0
batch = 0
for (X, y) in train_dataloader:
    if (torch.any(X.isnan())):
        print(f"Found on batch: {batch}")
        count+=1
    batch +=1


Found on batch: 11
Found on batch: 30


In [114]:
class MLP(nn.Module):
    def __init__(self, input_features, hidden_features, output_features):
        super().__init__()
        self.l1 = nn.Sequential(
            nn.Linear(input_features, hidden_features),
            nn.ReLU()
        )
        self.l2 = nn.Sequential(
            nn.Linear(hidden_features, 15),
            nn.ReLU()
        )
        self.l3 = nn.Linear(15, output_features)

    def forward(self, x):
        x = self.l1(x)
        x = self.l2(x)
        x = self.l3(x)
        return x

In [115]:
mlpmodel = MLP(8, 30, 1)

In [116]:
loss_fn = nn.MSELoss()

optimizer = torch.optim.SGD(params=mlpmodel.parameters(), lr=1.e-6)

In [117]:
EPOCHS = 1

mlpmodel.train()
for epoch in range(EPOCHS):
    total_loss = 0
    for batch, (X, y) in enumerate(train_dataloader):
        preds = mlpmodel(X)
        # print(f"Preds + {preds}")
        loss = loss_fn(torch.unsqueeze(y, 1), preds)
        total_loss+= loss
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        print(f"We're on batch: {batch} + Loss: {total_loss/batch}")


We're on batch: 0 + Loss: inf
We're on batch: 1 + Loss: 4.559865450100075
We're on batch: 2 + Loss: 3.3377658211308088
We're on batch: 3 + Loss: 2.929034021344297
We're on batch: 4 + Loss: 2.7387721866566372
We're on batch: 5 + Loss: 2.646381759667686
We're on batch: 6 + Loss: 2.583671104417694
We're on batch: 7 + Loss: nan
We're on batch: 8 + Loss: nan
We're on batch: 9 + Loss: nan
We're on batch: 10 + Loss: nan
We're on batch: 11 + Loss: nan
We're on batch: 12 + Loss: nan
We're on batch: 13 + Loss: nan
We're on batch: 14 + Loss: nan
We're on batch: 15 + Loss: nan
We're on batch: 16 + Loss: nan
We're on batch: 17 + Loss: nan
We're on batch: 18 + Loss: nan
We're on batch: 19 + Loss: nan
We're on batch: 20 + Loss: nan
We're on batch: 21 + Loss: nan
We're on batch: 22 + Loss: nan
We're on batch: 23 + Loss: nan
We're on batch: 24 + Loss: nan
We're on batch: 25 + Loss: nan
We're on batch: 26 + Loss: nan
We're on batch: 27 + Loss: nan
We're on batch: 28 + Loss: nan
We're on batch: 29 + Loss